In [21]:
import numpy as np
def cpca(X,d_): #X: list of (d_v,n)
    X = np.concatenate(X)
    X = X-X.mean(axis=1).reshape(-1,1)
    cov_X = X.dot(X.T)
    eigenval,eigenvec = np.linalg.eig(cov_X)
    eigenval = np.real(eigenval)
    eigenvec = np.real(eigenvec)    
    idx = np.argpartition(eigenval,-d_)[-d_:]
    select_vec = eigenvec[:,idx]
    out = select_vec.T.dot(X)
    return out



a = np.array([[1,2],[3,4]])
b = np.random.rand(3,4)
# np.concatenate([a,b])
a-a.mean(1)
b-b.mean(1).reshape(-1,1)
np.linalg.eig(b.dot(b.T))



(array([3.33189021, 0.2673703 , 0.10820184]),
 array([[ 0.57866893,  0.80676435,  0.11947192],
        [ 0.19772667,  0.00333957, -0.9802515 ],
        [ 0.79123096, -0.59086388,  0.15758632]]))

In [25]:
import scipy.io as sio
import numpy as np
def read_data(filename):
    data = sio.loadmat(filename)
    X = data['X']
    labels = data['Y']
    return X,labels


X,labels=read_data('datasets/data sets/MSRC-v1.mat')
print(X.shape)
X = [X[0,i].T for  i  in range(X.shape[1])]
cpca(X,100)


(1, 4)


array([[-1.37562826e+01,  3.97926279e+01, -3.70780104e+01, ...,
         5.66778439e+00, -3.40605646e+01,  2.92823300e+00],
       [-2.88209924e+01,  1.28707303e+01,  4.61312520e+01, ...,
        -3.59879011e+01, -3.36239870e+01,  1.12994373e+01],
       [ 7.76526491e+00, -6.12214855e+01, -4.28835566e+01, ...,
         3.10512348e+01,  8.11416255e+00, -2.92831900e+01],
       ...,
       [ 3.13730149e+03,  3.71985045e+03,  4.18338430e+03, ...,
         2.86582231e+03,  2.37181821e+03,  3.29955447e+02],
       [ 2.86748482e+03,  4.98876447e+03,  5.48622250e+03, ...,
         3.65420348e+03,  2.94284771e+03,  3.78493802e+03],
       [-2.57924737e+03, -1.34066475e+03, -1.41657946e+03, ...,
        -4.65623749e+03, -5.45172177e+03, -5.72210694e+03]])

In [ ]:
import tarfile
import os
from tqdm import tqdm
import numpy as np



def untar(fname, dirs):
    """
    解压tar.gz文件
    :param fname: 压缩文件名
    :param dirs: 解压后的存放路径
    :return: bool
    """
    try:
        t = tarfile.open(fname)
        t.extractall(path = dirs)
        return True
    except Exception as e:
        print(e)
        return False

# untar('datasets/msrcorid.tar.gz','./datasets/')
from PIL import Image
def read_imgs():
    path = 'datasets/msrcorid'
    filepaths = []
    img_arrays = []
    labels = []
    for root, dirs, files in tqdm(os.walk(path)):
        # print(dirnames)     
        for file in files:
            if(file.endswith('JPG')):                         
                file_path = os.path.join(root, file)
                img = np.asarray(Image.open(file_path))
                img_arrays.append(img.transpose((2,0,1)).reshape((3,-1,1)))
                labels.append(file[:3])   
                filepaths.append(file_path)           
       
    return img_arrays,labels
def color_moment(n,img_tensor):
    color_mean = img_tensor.mean(axis=1)
    temp= ((img_tensor-np.expand_dims(color_mean,axis = 1))**n).mean(axis=1)
    return temp**(1/n)

img_arrays,labels = read_imgs() #img_arrays = [(ch,n_pixel,1),...]
img_tensor = np.concatenate(img_arrays,axis = 2)
color_mean = img_tensor.mean(axis=1)
# color_moment(2,img_tensor)

